<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Hey-friends-:)" data-toc-modified-id="Hey-friends-:)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Hey friends :)</a></span></li><li><span><a href="#Compartment-Architecture" data-toc-modified-id="Compartment-Architecture-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Compartment Architecture</a></span></li><li><span><a href="#Data-Pre-processing" data-toc-modified-id="Data-Pre-processing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data Pre-processing</a></span><ul class="toc-item"><li><span><a href="#outflows_data" data-toc-modified-id="outflows_data-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>outflows_data</a></span></li><li><span><a href="#transitions_data" data-toc-modified-id="transitions_data-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>transitions_data</a></span></li><li><span><a href="#total_population_data" data-toc-modified-id="total_population_data-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>total_population_data</a></span></li><li><span><a href="#Uploading-our-data" data-toc-modified-id="Uploading-our-data-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Uploading our data</a></span></li></ul></li><li><span><a href="#Running-a-baseline" data-toc-modified-id="Running-a-baseline-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Running a baseline</a></span></li><li><span><a href="#Running-a-policy-scenario" data-toc-modified-id="Running-a-policy-scenario-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Running a policy scenario</a></span></li><li><span><a href="#Extensions" data-toc-modified-id="Extensions-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Extensions</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Outflows-data" data-toc-modified-id="Outflows-data-6.0.1"><span class="toc-item-num">6.0.1&nbsp;&nbsp;</span>Outflows data</a></span></li><li><span><a href="#Transitions-data" data-toc-modified-id="Transitions-data-6.0.2"><span class="toc-item-num">6.0.2&nbsp;&nbsp;</span>Transitions data</a></span></li><li><span><a href="#Total-population-data" data-toc-modified-id="Total-population-data-6.0.3"><span class="toc-item-num">6.0.3&nbsp;&nbsp;</span>Total population data</a></span></li></ul></li></ul></li></ul></div>

# Hey friends :)
This notebook will walk you thru running your first policy simulation (woooooo!). You should run each cell of code– starting with the imports under this text–working down the notebook sequentially. There will be text boxes like this one instructing you what you need to fill in to get stuff working. 

You're encouraged to experiment, visualize, and add cells as you please. You don't need to understand every line of code you're running, but obviously you can look up functions and stuff that did nifty things you want to be able to reproduce. DM Tori or Paco on Slack if you get stuck <3

In [1]:
import os
import sys
sys.path.insert(0, os.path.relpath('../../')) 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from super_simulation.super_simulation_factory import SuperSimulationFactory
from transition_table import TransitionTable
from spark_policy import SparkPolicy
from utils.spark_bq_utils import upload_spark_model_inputs
import pandas as pd
from functools import partial

pd.set_option('display.max_rows', 500)

# Compartment Architecture
The first step in Spark modeling is always to decide what compartment architecture you want to build around. In our case, we're gonna use this one:

**'pretrial' (shell compartment) --> 'prison' (full compartment) --> 'release' (full compartment)**

Make sure you use these three compartment names in your data processing!

# Data Pre-processing
We're skipping the early part for now, but you'll still need to get from "clean" data to the data inputs that the Spark model ingests. Feel free to run this cell without thinking about it too much.

In [2]:
raw_va_sentence_df = pd.read_csv(
    '../../state/VA/VA_data/unprocessed_va_historical_sentences_v2.csv',
    sep='\t'
)
raw_va_sentence_df['crime_type'] = raw_va_sentence_df['Offense Group'].ffill()
raw_va_sentence_df['offense_code'] = raw_va_sentence_df['VCC'].ffill()
raw_va_sentence_df['crime'] = raw_va_sentence_df['Off1VCC'].ffill()
raw_va_sentence_df['judge_id'] = raw_va_sentence_df['JudgeID'].ffill()
raw_va_sentence_df['sentence_type_code'] = raw_va_sentence_df['ActDisp'].ffill()
raw_va_sentence_df['effective_sentence_months'] = raw_va_sentence_df['effsent']
raw_va_sentence_df['fiscal_year'] = raw_va_sentence_df['FiscalYr'].ffill()
raw_va_sentence_df['life_sentence'] = raw_va_sentence_df['EffLif']
raw_va_sentence_df = raw_va_sentence_df.rename({'Off1Date':'offense_date'}, axis=1)
raw_va_sentence_df = raw_va_sentence_df[raw_va_sentence_df.sentence_type_code != 1]
raw_va_sentence_df = raw_va_sentence_df[raw_va_sentence_df.effective_sentence_months != 0]

raw_va_sentence_df = raw_va_sentence_df[~raw_va_sentence_df['crime_type'].str.contains('Total')]
raw_va_sentence_df = raw_va_sentence_df.drop(
    ['VCC', 'Offense Group', 'Off1VCC', 'JudgeID', 'ActDisp', 'effsent', 'FiscalYr', 'EffLif'],
    axis=1
)
raw_va_sentence_df.head()

,offense_date,OffLName,OffFName,crime_type,offense_code,crime,judge_id,sentence_type_code,effective_sentence_months,fiscal_year,life_sentence
0,5/20/2017,MOORE,KENNETH,ASSAULT,ASL1316,ASL1316F6,0,3.0,18.0,2018.0,0.0
1,4/25/2018,CLINE,MATTHEW,ASSAULT,ASL1316,ASL1316F6,0,3.0,10.0,2019.0,0.0
4,12/25/2016,MILLER,MARCUS,ASSAULT,ASL1316,ASL1316F6,10DB,3.0,7.0,2017.0,0.0
5,11/17/2018,SPRADLEY,CALEB,ASSAULT,ASL1316,ASL1316F6,10DB,3.0,7.0,2019.0,0.0
7,12/11/2016,JAMES,ERIC,ASSAULT,ASL1316,ASL1316F6,10KW,3.0,14.0,2017.0,0.0


We can see the first few rows of the dataset above, but let's take a slightly more comprehensive peak at what's in here before we move on! You can run the following cells without changing anything, but feel free to play with them anyway.

In [ ]:
# Always a good place to start!
raw_va_sentence_df.describe([0.05,0.25,0.5,0.75,0.95])

In [ ]:
# Combine first and last name into one full name and check the number of rows of data per person
raw_va_sentence_df['full_name'] = raw_va_sentence_df.OffFName + ' ' + raw_va_sentence_df.OffLName
sentences_per_person = raw_va_sentence_df.groupby('full_name').count().sort_values('OffLName', ascending=False)
plt.hist(sentences_per_person.OffLName, bins = 30)
plt.ylabel('number of people')
plt.xlabel('number of rows of data per person')
plt.yscale('log')

In [ ]:
# Cap sentence lengths at 25 years, then plot the sentence length distribution in months.
temp = raw_va_sentence_df.copy()
temp.loc[temp.effective_sentence_months > 12 * 25, 'effective_sentence_months'] = 12 * 25
plt.hist(temp.effective_sentence_months, bins=40)
plt.ylabel('number of people')
plt.xlabel('length of sentence in months')
# plt.yscale('log')

## outflows_data
Ok, time to process our data! First up is the outflows_data table. As a reminder, outflows_data counts the number of people transitioning historically into and throughout the system. We're keeping it simple for now, so we'll just use one historical average admission number. Here is what a final outflows_data table should look like:

In [ ]:
pd.DataFrame({
    'compartment': ['pretrial', 'pretrial'],
    'outflow_to': ['prison'] * 2, # this is the same as writing it out two times
    'crime_type': ['NA'] * 2,
    'time_step': [0,1],
    'total_population': [250, 233]
})

Okee you're up! Turn `raw_va_sentence_df` into an outflows_data table! Some important tips for this:
* Make sure your table has exactly the right columns with the right names when you're done. df.rename() is your friend!
* `time_step` is the number of steps away from the reference date you're at. So with a reference date of 2020.0 and a time_step of 1 year, 2021.0 would be time_step = 1. In our case, the reference_date is 2019.


In [ ]:
# Step 1: Use sub_df = df[*logical condition on column*] to get the subset of data for just 2019. 
#* your code here *

# Step 2: Use len(df) to calculate the number of people sentenced to prison time in 2019. 
#* your code here *

# Step 3: Create a one row dataframe with the datapoint we just derived that matches the columns of the example  
# in the cell above.
#* your code here *

# Step 4: Ensure the columns of our table have the correct types. In particular, make sure the `time_step` column
# has type int and `total_population` column has type float. You can use df.col_name = df.col_name.astype(type)
#* your code here *

Whoooo look at you go!!

## transitions_data
On to the transitions_data table. Transitions_data should have one row of data per full compartment (aka every one except pretrial) that captures the average length of stay (LOS) in that compartment. Here is what a final transitions table should look like:

In [ ]:
pd.DataFrame({
    'compartment': ['prison'] * 2 + ['release'] * 1,
    'outflow_to': ['release'] * 3,
    'crime_type': ['NA'] * 3,
    'compartment_duration': [48,156, 1],
    'total_population': [0.6, 0.4, 1]
})

Okee you're up! Turn `raw_va_sentence_df` into a transitions_data table! Some important tips for this:
* Make sure you make a copy of `raw_va_sentence_df` and mess with that. If you start changing the original you won't have anything to build the other data inputs with.
* Make sure your table has exactly the right columns with the right names when you're done. df.rename() is your friend!
* Our time steps are one year, make sure the units of your `compartment_duration` column are correct.


In [ ]:
# Step 1: Use df.col_name.mean() on the appropriate column to get the average LOS across all sentences.

#* your code here *

# Step 2: Convert units using avg_LOS = avg_LOS * scale_factor

#* your code here *

# Step 3: You've now finished the transitions data for the prison compartment, but we still need transitions data
# for the release compartment. For our purposes, we're gonna assume no recidivism, which means people can just go
# from release back to release (just like the example in the cell above).
# Create a dataframe with two rows, one for the prison LOS and one for release back to itself, that matches 
# the columns of the example in the cell above. Since there's only one row per compartment, you can just
# set the total_population column to 1 for both.

#* your code here *

# Step 4: Ensure the columns of our table have the correct types. In particular, make sure the `compartment_duration`
# and `total_population` columns both have type float.

#* your code here *

Killin it!! Now let's take a look at what we just made and sanity-check that it looks reasonable:

In [ ]:
transitions_data

## total_population_data
Last but not least: the total_population table. You'll have to go grab data for this one yourself, the data you need is on page 37 of this [report](https://vadoc.virginia.gov/media/1623/vadoc-financial-annual-mis-report-2020.pdf).

We just need the total population for the year 2019, with which we'll make our final data input table. total_population_data should look like this:

In [ ]:
pd.DataFrame({
    'compartment': ['prison'] * 4,
    'time_step': range(3,7),
    'total_population': [2500, 2800, 3030, 2820]
})

Okee you're up! Turn the table from the DOC report into a `total_population_data` table! Some important tips for this:
* Make sure your table has exactly the right columns with the right names when you're done.
* `time_step` is the number of steps away from the reference date you're at. So with a reference date of 2020.0 and a time_step of 1 year, 2022.0 would be time_step = 2. In our case, the reference_date is 2019.

In [ ]:
# Step 1: Input the data from the DOC report into a pandas DataFrame. Feel free to start with the example table
# above if that's easier. 

#* your code here *

# Step 2: Ensure the columns of our table have the correct types. In particular, make sure the `time_step` column has
# type int and `total_population` column has type float. You can use df.col_name = df.col_name.astype(type)

#* your code here *

## Uploading our data
The final step in pre-processing is to upload our data to Google BigQuery. If this next cell gives you a permission error, you need to go set up your Google Cloud access–follow these steps:
1. Work through just the installation section of [this page](https://cloud.google.com/sdk/docs/quickstart). Make sure you log in with your Stanford email.
2. Open a terminal window and navigate to your Github repository folder. Run the following commands:
3. `gcloud auth application-default login` --> This will prompt you to log in to your google cloud account
4. `gcloud config set project recidiviz-staging`

Once you're set up, you can try the upload again.


In [ ]:
# STEP 1: pick a `simulation_tag` for your simulation. "paco_parole_test" would be a reasonable example...
simulation_tag = ""


upload_spark_model_inputs(
    "recidiviz-staging",
    simulation_tag,
    outflows_data,
    transitions_data,
    total_population_data,
    '../../state/VA/2022/test_configurations/walkthru_user_inputs.yaml',
)

# Running a baseline
We're ready to build a simulation now! First step is initializing a simulation object, called a SuperSimulation. When we initialize it we pass in the filepath of our YAML configuration, so before you continue you need to go to that file, located in `recidiviz-data/recidiviz/calculator/modeling/population_projection/state/VA/2022/test_configurations`, and change the simulation_tag to match the one you chose above.

In [ ]:
# Don't worry about any warnings. As long as this doesn't fail, you're in business
spark_sim = SuperSimulationFactory.build_super_simulation(
    '../../state/VA/2022/test_configurations/walkthru_user_inputs.yaml'
)

Now we're going to run a baseline simulation. This is an oppportunity to make sure the data we generated is valid, and to sanity-check the basic dynamics it generates. You should just be able to run the code below, but if you feel like it you can also include `release` in the `display_compartments` list to see that population evolve too.
*Make sure you pause to think about the graph this generates*. Do you believe this is actually close to reality? Is there something off, and if so what might be causing it?

In [ ]:
#This can take several minutes to run. You'll see six checkpoints printed out along the way before it finishes.
display_compartments = ['prison']
spark_sim.simulate_baseline(display_compartments)

# Running a policy scenario
I've filled out a lot of the code for you here, but once you finish out the missing pieces, definitely play around with the stuff I did for you. The whole appeal of the Spark model is how easy it is to test out different policy scenarios once you get the baseline running.

Without worrying about all the details, the main thing to understand here is that a policy function transforms the transitions data of a given compartment. By applying one or multiple per compartment that is affected in the policy scenario, we can simulate the effects of the reform in question. `policy_list` is thus just a list of policy functions that gets passed into the simulation so it knows what to change in the policy scenario.

Some important pointers:
* A SparkPolicy specifies both compartment and subgroup. If you want a policy to apply to every subgroup you need one SparkPolicy per subgroup!
* Retroactivity determines whether or not a policy applies to people who are already incarcerated. Try toggling it on and off and see what happens!
* Order matters! If policy A shortens sentences by 50% and policy B shortens them by 1 year, the policy_list [A,B] will be different from the policy_list [B,A]

In [ ]:
# To start, let's model enabling parole by shortening the fraction of their sentence that people must serve.
# Currently, they cannot serve less than 85%. Let's suppose that this policy will bring that number down to 60%,
# and do so for everyone in prison. Think about how you should shorten prison transitions to capture that change, 
# then pick the appropriate parameters below!

REDUCTION_TYPE = ?? # This should either be '*' or '+'. If you make it '+', it will change compartment_duration L to
                    # L - LOS_REDUCTION. If you make it '*', it will change L to L * (1 - LOS_REDUCTION)
LOS_REDUCTION = ?? # This parameter determines how much shorter compartment duration gets for affected individuals.
AFFECTED_FRACTION = ?? # This parameter determines the fraction of the compartment population affected by the change.
RETROACTIVE = ?? # This should either be True or False

#### Should not have to change things below this line for this cell ####

def apply_reinstated_parole():
    return partial(TransitionTable.apply_reduction, 
                   reduction_df=pd.DataFrame({
                       'outflow': [OUTFLOW],
                       'reduction_size': [LOS_REDUCTION], 
                       'affected_fraction': [AFFECTED_FRACTION]
                   }),
                   reduction_type=REDUCTION_TYPE,
                   retroactive=RETROACTIVE)

# Note that I've created for you in this example, you don't have to build it yourself just yet.
policy_list = [SparkPolicy(policy_fn=apply_reinstated_parole(),
                                           spark_compartment='prison',
                                           sub_population={'crime_type': 'NA'},
                                           policy_ts=24,
                                           apply_retroactive=RETROACTIVE)]
    
test_results = spark_sim.simulate_policy(policy_list, 'prison')

How do those results look? Does this match your expectation for the magnitude of this policy's impact?

In [ ]:
# Now let's try mixing it up a bit. Now I want you to try changing the policy so it only applies to 
# non-violent offenders. Suppose 60% of people in prison are non-violent.

LOS_REDUCTION = ??
AFFECTED_FRACTION = ??
REDUCTION_TYPE = ??
RETROACTIVE = ??
# If you're unsure what a variable will do, try running it with different values and see how the results change

def apply_reinstated_parole():
    return partial(TransitionTable.apply_reduction, 
                   reduction_df=pd.DataFrame({
                       'outflow': [OUTFLOW],
                       'reduction_size': [LOS_REDUCTION], 
                       'affected_fraction': [AFFECTED_FRACTION]
                   }),
                   reduction_type=REDUCTION_TYPE,
                   retroactive=RETROACTIVE)

policy_list = [SparkPolicy(policy_fn=apply_reinstated_parole(),
                                           spark_compartment='prison',
                                           sub_population={'crime_type': 'NA'},
                                           policy_ts=24,
                                           apply_retroactive=RETROACTIVE)]
    
test_results = spark_sim.simulate_policy(policy_list, 'prison')

How do these results compare to your previous scenario? Is it more or less? By how much? Can you think of a way to validate that that result is reasonable?

# Extensions
Try these to get a feel for how each of the things you worked on affects the final result:

### Outflows data
What happens to the simulation when you change the magnitude of outflows data? What about if you added another year (or years) of data?

### Transitions data
What happens to the simulation if you change the `compartment_duration` of the transitions data? What about if you add another row of data (e.g. everything else the same but change the `compartment_duration` for the second one)? What happens if you make release feed to prison instead of itself? Why?

### Total population data
What happens to the simulation if you change the magnitude of the population data? Does it change the beginning and the end of the simulation differently? What happens if you pass an empty table (i.e. `total_population_data = pd.DataFrame()`)?